In [6]:
import nltk
import json
import pandas as pd
import gzip
import string
import math
import time
import numpy as np
from nltk.stem.porter import PorterStemmer
porter =PorterStemmer()

In [7]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [8]:
# 确定数据集
GC5 = getDF('Gift_Cards_5.json.gz')
mGC = getDF('meta_Gift_Cards.json.gz')
GC5['reviewText']=GC5['reviewText'].astype(str) # reviewText的部分格式是float,将其转换为str格式

In [14]:
# 针对数据集GC5的reviewText域
class IRserach1(object):
    def __init__(self, GC5, k):
        self.__desindex = [0]*k # 存储topk对应的k个docID
        self.__query = '' # 存储用户查询的query
        self.__t = 0 # 计算运行时间
        self.__token = [[]for i in range(len(GC5))] # 计算document的token列表
        self.__term = set() # 计算term集合
        self.__hashtable = {} # 存储倒排索引表
        self.__idf = [0 for i in range(2980)] # 这里2980就是size(term)
        self.__tf_idf = {} # tf-idf数组
        self.__simcoslist = [] # 余弦相似度列表
        
    # 读取query
#     def getquery(self):
#         query = input()
#         return query

    # stemming 操作
    def stem(self, GC5):
        t1 = time.time()
        # 处理GC5数据集
        for i in range(len(GC5)):
            exclude = set(string.punctuation)
            GC5.iloc[i,7] = ''.join(ch for ch in GC5.iloc[i,7] if ch not in exclude) # 去掉标点

            temp1 = GC5.iloc[i,7]
            GC5.iloc[i,7]=porter.stem(temp1) # reviewText
            
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    # 计算document的token列表和term集合
    def gettokenterm(self, GC5):
        t1 = time.time()
        for index,file in enumerate(GC5):
            seg_res = file.split()
            self.__token[index].extend(seg_res)
            self.__term.update(set(seg_res))
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    # 计算GC5的review域的倒排索引表
    def getInvertedindex(self):
        t1 = time.time()
        for index, li in enumerate(self.__token):
            for word in li:
                if word in self.__hashtable:
                    self.__hashtable[word].append(index+1)
                    self.__hashtable[word][0] += 1
                else:
                    self.__hashtable[word] = [1, index+1]
        self.__hashtable = dict(sorted(self.__hashtable.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True))
        
        if(len(self.__term) != len(self.__idf)):
            self.__idf = [0 for i in range(len(self.__term))]
#         print(len(self.__term))
#         print(self.__hashtable)
        t2 = time.time()
        self.__t = self.__t + t2 - t1

    
    # 计算每个document的tf-idf向量
    def gettf_idf(self):
        t1 = time.time()
        for i, word in enumerate(self.__term):
            temp = list(word in line for line in self.__token).count(True) #df就是出现word的文档数  
            self.__idf[i] = math.log(len(self.__token)/(temp), 7) #计算每个word的idf值 
            
        for index, line in enumerate(self.__token): # n
            tf = [0 if line.count(term) == 0
                  else 1+ math.log(line.count(term), 7)
                  for term in self.__term]
            self.__tf_idf[index] = np.multiply(tf, self.__idf)
            
        t2 = time.time()
        self.__t = self.__t + t2 - t1
    
    def getsmicos_li(self):
        t1 = time.time()
        # 先算出query的tf-idf向量
#         line = self.getquery()
        self.__query = input()
        line = str(self.__query.split())
        
        line = porter.stem(line)
        print(line)
#         print(line)
        tf = [0 if line.count(term) == 0 # 为了防止len(doc)为0报错, 所以先进行判断
          # 但是这样处理答案全是0, 故考虑取对数处理进行放大放大
            else 1+ math.log(line.count(term), 7)
            for term in self.__term] 
        tfidf_query = np.multiply(tf, self.__idf)
#         print(tfidf_query)
        v1 = tfidf_query # 这里v1作为query
        for j in range(len(self.__token)):
            v2 = self.__tf_idf[j]
            # 使用内积和模长计算余弦相似度
            sim_cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
            self.__simcoslist.append(sim_cos)
        
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        return self.__simcoslist
#         print(self.__simcoslist)
    # topk代码
    def sift(self, GC5, low, high):
        t1 = time.time()
        i = low
        j = 2 * i + 1
        tmp = GC5[low]
        while j <= high:
            if j + 1 <= high and GC5[j+1] < GC5[j]:
                j = j + 1
            if GC5[j] < tmp:
                GC5[i] = GC5[j]
                i = j
                j = 2 * i + 1
            else:
                break
            GC5[i] = tmp

        t2 = time.time()
        self.__t = self.__t + t2 - t1

    def getIndex(self, heap,li_cossim, k):
        t1 = time.time()
        
        for j in range(k): # 针对前k个最大的相似度值 j=0
            for i in range(len(li_cossim)): # 遍历寻找对应的下标
                if li_cossim[i] == heap[j]:
                    self.__desindex[j] = (i)
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    def priRes(self, heap, k, dataset):
        
        t1 = time.time()
        print("the query is:\n", self.__query)
        print("***#######################***")
        print("the top{} most similar documents are:\n".format(k))
        if np.isnan(heap[0]): # 如果相关性最高的相似余弦也为nan, 说明检索失败
            print("search fail")
        else:
            for i in range(k):
                print("reviewText: ", dataset.iloc[self.__desindex[i]][7]) #7是reviewText特征
                print("余弦相似度为: ", heap[i]) 
                
                print("reviewTime: \n", dataset.iloc[self.__desindex[i]][2]) # reviewTime
                print("reviewerName: \n", dataset.iloc[self.__desindex[i]][6]) # reviewerName
                print("summary: \n", dataset.iloc[self.__desindex[i]][8]) # summary
                print("-----------**********-----------")
        print("***#######################***\n")
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        return True


    #解决topk问题的主体函数
    def topk(self, li, k, data):
        
        heap = li[0:k]
        for i in range((k-2)//2, -1, -1):
            self.sift(heap, i, k-1)
        # 1.建堆
        for i in range(k, len(li)-1):
            if li[i] > heap[0]:
                heap[0] = li[i]
                self.sift(heap, 0, k-1)

        # 2.遍历
        for i in range(k-1, -1, -1):
            heap[0], heap[i] = heap[i], heap[0]
            self.sift(heap, 0, i - 1)

        # 3.得到top5的下标
        desindex = self.getIndex(heap,li, k)

        # 4.输出结果
        self.priRes(heap, k, data)

        # 计时结束

        print("总计花费时间为: \n",self.__t, 's')

        return True

In [15]:
def startsearch1():
    test = IRserach1(GC5['reviewText'], 15)
    test.stem(GC5)
    sample = list(GC5['reviewText'].values)
    test.gettokenterm(sample)
    test.getInvertedindex()
    test.gettf_idf()
    smicoslist = test.getsmicos_li()
    GC5ori = getDF('Gift_Cards_5.json.gz')
    test.topk(smicoslist,15,GC5ori)

In [16]:
# 数据集GC5的reviewText域进行query搜索:
startsearch1()

apple
['apple']
the query is:
 apple
***#######################***
the top15 most similar documents are:

reviewText:  My wife absolutely loves starbucks.  This allows her to load funds on her app on the phone.
余弦相似度为:  0.20626452281095106
reviewTime: 
 09 20, 2016
reviewerName: 
 Eric Beckman
summary: 
 Loves it.
-----------**********-----------
reviewText:  Just didn't  know  it was for a girl  gift card, and I was  looking  around  Christmas tree  gifts  cards and type  in right  it was for a boy and I did the e card so he could use it then.
余弦相似度为:  0.17763056439166441
reviewTime: 
 01 10, 2018
reviewerName: 
 Amazon Customer
summary: 
 Just didn't know it was for a girl gift card ...
-----------**********-----------
reviewText:  This is the easy part. Its the ppl at safeway who have no idea how to redeem them. I learned they have to press the eft button in order to process the card correctly otherwise it says override needed and that doesnt work either. Also if u don't use it all 

E:\conda_temp\ipykernel_18132\1225428629.py:99: RuntimeWarning: invalid value encountered in double_scalars
  sim_cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))


In [206]:
# test = IRserach(GC5['reviewText'], 15)
# test.stem(GC5)
# sample = list(GC5['reviewText'].values)
# test.gettokenterm(sample)

In [207]:
# test.getInvertedindex()
# test.gettf_idf()

In [208]:
# smicoslist = test.getsmicos_li()

another great gift
['another', 'great', 'gift']


E:\conda_temp\ipykernel_13544\67666115.py:92: RuntimeWarning: invalid value encountered in double_scalars
  sim_cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))


In [209]:
# test.topk(smicoslist,5)

the query is:
 another great gift
***#######################***
the top5 most similar documents are:

1.0 another great gift
0.8712716514710099 another great christmas gift
0.5663303522051254 another favorite item
0.5445257033000609 another favorit
0.5334106020971117 and another great chiatmas gift
***#######################***

总计花费时间为: 
 15.853684425354004 s


[1.0,
 0.8712716514710099,
 0.5663303522051254,
 0.5445257033000609,
 0.5334106020971117]

In [112]:
# 针对数据集GC5的reviewText域
class IRserach2(object):
    def __init__(self, GC5, k):
        self.__desindex = [0]*k # 存储topk对应的k个docID
        self.__query = '' # 存储用户查询的query
        self.__t = 0 # 计算运行时间
        self.__token = [[]for i in range(len(GC5))] # 计算document的token列表
        self.__term = set() # 计算term集合
        self.__hashtable = {} # 存储倒排索引表
        self.__idf = [0 for i in range(2980)] # 这里2980就是size(term)
        self.__tf_idf = {} # tf-idf数组
        self.__simcoslist = [] # 余弦相似度列表
        
    # 读取query
#     def getquery(self):
#         query = input()
#         return query

    # stemming 操作
    def stem(self, GC5):
        t1 = time.time()
        # 处理GC5数据集
        for i in range(len(GC5)):
            exclude = set(string.punctuation)
            GC5.iloc[i,7] = ''.join(ch for ch in GC5.iloc[i,7] if ch not in exclude) # 去掉标点

            temp1 = GC5.iloc[i,7]
            GC5.iloc[i,7]=porter.stem(temp1) # reviewText
            
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    # 计算document的token列表和term集合
    def gettokenterm(self, GC5):
        t1 = time.time()
        for index,file in enumerate(GC5):
            seg_res = file.split()
            self.__token[index].extend(seg_res)
            self.__term.update(set(seg_res))
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    # 计算GC5的review域的倒排索引表
    def getInvertedindex(self):
        t1 = time.time()
        for index, li in enumerate(self.__token):
            for word in li:
                if word in self.__hashtable:
                    self.__hashtable[word].append(index+1)
                    self.__hashtable[word][0] += 1
                else:
                    self.__hashtable[word] = [1, index+1]
        self.__hashtable = dict(sorted(self.__hashtable.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True))
        
        if(len(self.__term) != len(self.__idf)):
            self.__idf = [0 for i in range(len(self.__term))]
#         print(len(self.__term))
#         print(self.__hashtable)
        t2 = time.time()
        self.__t = self.__t + t2 - t1

    
    # 计算每个document的tf-idf向量
    def gettf_idf(self):
        t1 = time.time()
        for i, word in enumerate(self.__term):
            temp = list(word in line for line in self.__token).count(True) #df就是出现word的文档数  
            self.__idf[i] = math.log(len(self.__token)/(temp), 7) #计算每个word的idf值 
            
        for index, line in enumerate(self.__token): # n
            tf = [0 if line.count(term) == 0
                  else 1+ math.log(line.count(term), 7)
                  for term in self.__term]
            self.__tf_idf[index] = np.multiply(tf, self.__idf)
            
        t2 = time.time()
        self.__t = self.__t + t2 - t1
    
    def getsmicos_li(self):
        t1 = time.time()
        # 先算出query的tf-idf向量
#         line = self.getquery()
        self.__query = input()
        line = list(self.__query.split())
        
        
        
        print(line)
#         print(line)
        tf = [0 if line.count(term) == 0 # 为了防止len(doc)为0报错, 所以先进行判断
          # 但是这样处理答案全是0, 故考虑取对数处理进行放大放大
            else 1+ math.log(line.count(term), 7)
            for term in self.__term] 
        tfidf_query = np.multiply(tf, self.__idf)
#         print(tfidf_query)
        v1 = tfidf_query # 这里v1作为query
        for j in range(len(self.__token)):
            v2 = self.__tf_idf[j]
            # 使用内积和模长计算余弦相似度
            sim_cos = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
            self.__simcoslist.append(sim_cos)
        
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        return self.__simcoslist
#         print(self.__simcoslist)
    # topk代码
    def sift(self, GC5, low, high):
        t1 = time.time()
        i = low
        j = 2 * i + 1
        tmp = GC5[low]
        while j <= high:
            if j + 1 <= high and GC5[j+1] < GC5[j]:
                j = j + 1
            if GC5[j] < tmp:
                GC5[i] = GC5[j]
                i = j
                j = 2 * i + 1
            else:
                break
            GC5[i] = tmp

        t2 = time.time()
        self.__t = self.__t + t2 - t1

    def getIndex(self, heap,li_cossim, k):
        t1 = time.time()
        
        for j in range(k): # 针对前k个最大的相似度值 j=0
            for i in range(len(li_cossim)): # 遍历寻找对应的下标
                if li_cossim[i] == heap[j]:
                    self.__desindex[j] = (i)
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        
    def priRes(self, heap, k, dataset):
        
        t1 = time.time()
        print("the query is:\n", self.__query)
        print("***#######################***")
        print("the top{} most similar documents are:\n".format(k))
        if np.isnan(heap[0]): # 如果相关性最高的相似余弦也为nan, 说明检索失败
            print("search fail")
        else:
            for i in range(k):
                print("reviewText: ", dataset.iloc[self.__desindex[i]][7]) #7是reviewText特征
                print("余弦相似度为: ", heap[i]) 
                
                print("reviewTime: \n", dataset.iloc[self.__desindex[i]][2]) # reviewTime
                print("reviewerName: \n", dataset.iloc[self.__desindex[i]][6]) # reviewerName
                print("summary: \n", dataset.iloc[self.__desindex[i]][8]) # summary
                print("-----------**********-----------")
        print("***#######################***\n")
        t2 = time.time()
        self.__t = self.__t + t2 - t1
        return True


    #解决topk问题的主体函数
    def topk(self, li, k, data):
        
        heap = li[0:k]
        for i in range((k-2)//2, -1, -1):
            self.sift(heap, i, k-1)
        # 1.建堆
        for i in range(k, len(li)-1):
            if li[i] > heap[0]:
                heap[0] = li[i]
                self.sift(heap, 0, k-1)

        # 2.遍历
        for i in range(k-1, -1, -1):
            heap[0], heap[i] = heap[i], heap[0]
            self.sift(heap, 0, i - 1)

        # 3.得到top5的下标
        desindex = self.getIndex(heap,li, k)

        # 4.输出结果
        self.priRes(heap, k, data)

        # 计时结束

        print("总计花费时间为: \n",self.__t, 's')

        return True

In [113]:
def startsearch2():
    test = IRserach1(mGC['title'], 15)
    test.stem(mGC)
    sample = list(mGC['title'].values)
    test.gettokenterm(sample)
    test.getInvertedindex()
    test.gettf_idf()
    smicoslist = test.getsmicos_li()
    mGCori = getDF('meta_Gift_Cards.json.gz')
    test.topk(smicoslist,15, mGCori)

In [114]:
# 数据集mGC的title域进行query搜索:
startsearch2()

gift card
['gift', 'card']
the query is:
 gift card
***#######################***
the top15 most similar documents are:

reviewText:  by
    
    Apple
余弦相似度为:  0.7289993032881944
reviewTime: 
 []
reviewerName: 
 
summary: 
 ['Starbucks Gift Card 25', 'Starbucks']
-----------**********-----------
reviewText:  
余弦相似度为:  0.0
reviewTime: 
 ['Before you click to continue, you should know this isn&#x2019;t just a gift card. It&#x2019;s carte blanche for someone to pursue the dreams they have for their home. A wallet-sized dose of inspiration that grants them permission to improve and enhance. Just thought you should know. Now click away.']
reviewerName: 
 
summary: 
 []
-----------**********-----------
reviewText:  
余弦相似度为:  0.0
reviewTime: 
 ['Before you click to continue, you should know this isn&#x2019;t just a gift card. It&#x2019;s carte blanche for someone to pursue the dreams they have for their home. A wallet-sized dose of inspiration that grants them permission to improve and enhan